In [3]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, qty)
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    # load data.
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # shuffle an index vector
    idx = Vector(1:size(pts_pcs,1))
    rng = MersenneTwister(1234)
    idx = shuffle(rng, idx)
    
    # shuffle input pts and convert them to spherical
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]))

    # generate scan data
    scan = scan_tour(sph)
    
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
    
    # save
    writecsv("scan1.csv", scan)
end

test_transforms()
test_scan()

8.881784197001252e-16

(12146, 3)


In [4]:
data = readcsv("50_Holes_FrontLeft.csv");
pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

# shuffle an index vector
idx = Vector(1:size(pts_pcs,1))
rng = MersenneTwister(1234)
idx = shuffle(rng, idx)

# shuffle input pts and convert them to spherical
sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]));

In [5]:
# Modeling the problem as a TSP
max_vel = pi/4 # just a dummy variable for how fast the LP can change azimuth and elevation (Doesn't affect the solution)

using JuMP, Gurobi
using NamedArrays
n = 20 # number of points. 51 may cause my mac to explode.
m = Model(solver=GurobiSolver(OutputFlag=0))
@variable(m, x[1:n, 1:n], Bin)
@variable(m, 1 <= u[1:n] <= n, Int)

# each node must be touched
for i in 1:n
    cons = @constraint(m, sum(x[i,j] for j in 1:n) == 1)
end

for j in 1:n
    cons = @constraint(m, sum(x[i,j] for i in 1:n) == 1)
end

# no self loops
for i in 1:n
    cons = @constraint(m, x[i, i] == 0)
end

# MTZ Constraint
for i in 1:n
    for j in 2:n
        @constraint(m, u[i] - u[j] + n*x[i, j] <= n-1)
    end
end

@objective(m, Min, sum(max_vel*x[i,j]*abs(sph[i, 2] - sph[j, 2])
        + max_vel*x[i,j]*abs(sph[i, 3] - sph[j, 3]) for i in 1:n, j in 1:n));

@time solve(m)

display(getobjectivevalue(m))
x_opt = getvalue(x)

display(x_opt)

2.678287857426413

20×20 Array{Float64,2}:
 -0.0   0.0  -0.0  -0.0  -0.0   0.0  …  -0.0   0.0   0.0   1.0  -0.0   0.0
  0.0  -0.0  -0.0  -0.0  -0.0   1.0     -0.0  -0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0   0.0   1.0  -0.0
 -0.0  -0.0  -0.0  -0.0   1.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0      1.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0   1.0  -0.0  -0.0   0.0
 -0.0  -0.0  -0.0   1.0   0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   0.0
  0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0   0.0
  0.0   1.0  -0.0  -0.0  -0.0  -0.0     -0.0   0.0  -0.0  -0.0  -0.0   0.0
  1.0   0.0  -0.0  -0.0  -0.0   0.0     -0.0   0.0   0.0   0.0  -0.0   0.0
  0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0   1.0  -0.0  -0.0  -0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0     -0.0   0.0   0.0  -0.0  -0.0   1.0
 -0.0  -0.0   0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 

Academic license - for non-commercial use only
 15.070769 seconds (1.56 M allocations: 80.850 MiB)


In [14]:
scanned = []
i = 1
count = 1
for count in 1:n
    for j in 1:n
        if(x_opt[i, j] > 0)
            push!(scanned, j)
            i = j
            break
        end
    end
end

In [15]:
# subtour check
# scanned = []
# i = 1
# flag = true
# while(length(scanned)<n)
#     for j in 1:n
#         if(x_opt[i, j] > 0)
#             push!(scanned, j)
#             i = j
#             continue
#         end
#     end
#     if length(scanned) > length(unique(scanned))
#         print("Gotcha!")
#         print(i)
#     end
# end

In [16]:
length(unique(scanned))

20

In [17]:
length(scanned)

20

In [18]:
for j in scanned
    println("Scan feature $j")
end

Scan feature 10
Scan feature 15
Scan feature 5
Scan feature 4
Scan feature 7
Scan feature 8
Scan feature 17
Scan feature 6
Scan feature 2
Scan feature 9
Scan feature 11
Scan feature 16
Scan feature 13
Scan feature 19
Scan feature 3
Scan feature 20
Scan feature 12
Scan feature 14
Scan feature 18
Scan feature 1


In [9]:
ordered_features = zeros(n, 3)
i = 1
while(i<=n)
    j = scanned[i]
    ordered_features[i, :] = sph[j, :]
    i += 1
end

In [10]:
ordered_features

20×3 Array{Float64,2}:
 2.61359  1.78966  -2.04956
 2.04919  1.66696  -2.18631
 2.05449  1.63709  -2.18245
 2.07666  1.49795  -2.20861
 2.40443  1.44661  -2.10753
 2.48488  1.35118  -2.15744
 2.14786  1.35053  -2.19964
 2.63236  1.27712  -2.20449
 2.64078  1.2668   -2.20451
 2.33109  1.27382  -2.26356
 2.65872  1.26153  -2.38886
 2.546    1.24495  -2.33904
 2.12386  1.57002  -2.28567
 2.19587  1.59723  -2.31582
 2.51207  1.73556  -2.43765
 2.47795  2.04658  -2.52153
 2.35255  2.08092  -2.46887
 2.61918  2.26286  -2.2184 
 2.13695  1.87068  -2.20849
 2.62107  1.80213  -2.04955

In [11]:
# generate scan data
scan = scan_tour(ordered_features)

# back to pcs
scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
println(size(scan))

# save
writecsv("kartik_scan.csv", scan)

(1596, 3)


In [12]:
scan

1596×3 Array{Float64,2}:
 0.824679  0.735608  0.932535
 0.823668  0.742447  0.937591
 0.82268   0.749292  0.942631
 0.821716  0.756142  0.947653
 0.820774  0.762998  0.952658
 0.819856  0.769859  0.957646
 0.81896   0.776725  0.962616
 0.818088  0.783597  0.96757 
 0.817238  0.790473  0.972506
 0.816412  0.797355  0.977425
 0.815609  0.804241  0.982326
 0.814829  0.811133  0.98721 
 0.814073  0.818028  0.992077
 ⋮                           
 0.81212   0.806042  0.892421
 0.813177  0.79965   0.892989
 0.814252  0.793256  0.893564
 0.815344  0.786859  0.894146
 0.816454  0.78046   0.894734
 0.81758   0.774058  0.895329
 0.818724  0.767654  0.895931
 0.819884  0.761248  0.89654 
 0.821062  0.754839  0.897155
 0.822258  0.748429  0.897777
 0.82347   0.742016  0.898406
 0.8247    0.735601  0.899041